In [1]:
%display latex
from pylie import *
from pprint import pprint 
from collections.abc import Iterable
import functools
from operator import mul, sub
from pprint import pprint
from IPython.core.debugger import set_trace


In [2]:
def reduce(e1: Differential_Polynomial,e2: Differential_Polynomial, context:Context)->Differential_Polynomial:
    assert e2.is_monic()
    def _order (der):
        if der != 1:
            ## XXX: user pylie namespace
            return order_of_derivative(der)
        else :
            return [0]*len(context._independent)
    def func(e):    
        try:
            return e.operator().function()
        except AttributeError:
            return e.operator()   

    def _reduce (e, ld):
        e2_order = _order (ld)
        for t in e._p:
            d = t._d
            c = t._coeff
            if func(ld) != func(d):
                continue
            e1_order = _order (d)
            dif = [a-b for a, b in zip (e1_order, e2_order)]
            if all (map (lambda h: h == 0, dif)) :
                return Differential_Polynomial (e1.expression() - e2.expression() * c, context)
            elif all (map (lambda h: h >= 0, dif)):         
                variables_to_diff = []
                for i in range (len(context._independent)):
                    if dif [i] != 0:
                        variables_to_diff.extend ([context._independent[i]]*abs(dif[i]))      
                return Differential_Polynomial (e1.expression()-c*diff(e2.expression(), *variables_to_diff), context)
            else:
                pass
        return e

    _e1 = None
    while True:
        #set_trace()
        _e1 = _reduce (e1, e2.Lder())        
        if bool(_e1 == e1):
            return _e1
        e1 = _e1
        

In [3]:
def Reorder (S, context, ascending = False):
    res=sorted(S)
    if ascending :
        res.reverse()
    return res
def reduceS (e:Differential_Polynomial, S:list, context)->Differential_Polynomial:
    S= Reorder (S, context, ascending = True)
    reducing = True
    gen = (_ for _ in S)
    while reducing:
        for p in gen:
            enew = reduce (e, p, context)
            if enew == e:
                reducing = False
            else:
                e = enew
                gen = (_ for _ in S)
                reducing = True
    return enew

In [4]:
#indeterminates = (x, y) = var("x y")
#R=PolynomialRing(QQ, indeterminates)
#z  = function ("z")(*indeterminates)
#ctx = Context((z,), indeterminates)
#print("*"*70)
#e1 = diff(z,y)-x**2/y**2*diff(z,x)-(x-y)*z(x,y)/y**2
#e1_dp = Differential_Polynomial (e1, ctx)
#print (e1_dp)
#print("*"*70)
#f1 = diff(z, x) + z(x,y)/x
#f1_dp = Differential_Polynomial (f1, ctx)
#f1_dp.is_monic()
#print (f1_dp)
#print("*"*70)
#f2 = diff(z, y) + z(x,y)/y
#f2_dp = Differential_Polynomial (f2, ctx)
#print (f2_dp)
#r1 = reduce (e1_dp, f1_dp, ctx)
#r1
#r1._orig
#r2 = reduce (r1, f2_dp, ctx)

In [5]:
def Lfunc(e):
    return e

In [6]:
def Lcml(e):
    '''Least common left multiply, p.23'''
    return 0

In [7]:
def Lcm(e):
    return 0

In [8]:
def CompleteSystem(S, context):
    return S

In [9]:
def FindIntegrableConditions(S, context):
    return S

In [10]:
def ReduceIntegrabilityConditions(S, context):
    return S

In [11]:
var ("x y")
z = function("z")(x,y)
w = function("w")(x,y)
ctx = Context((w,z), (x,y), MatrixOrder.Mgrlex)
f1 =diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w(x,y)/y
f1.show()

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

In [12]:
f2=diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x
f2.show()

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

In [13]:
f3 = diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2
f3.show()

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

In [14]:
f4 = diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w(x,y)/(2*y**2)
f4.show()

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

In [15]:
f5 =diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w(x,y)/(y**2)
f5.show()

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

In [16]:
#f2a=f2-(y*f1/x);f2a.expand().show()

In [17]:
f1dp=Differential_Polynomial(f1, context=ctx)
f2dp=Differential_Polynomial(f2, context=ctx)
f3dp=Differential_Polynomial(f3, context=ctx)
f4dp=Differential_Polynomial(f4, context=ctx)
f5dp=Differential_Polynomial(f5, context=ctx)

In [18]:
ro = Reorder ([f1dp, f2dp, f3dp, f4dp, f5dp], ctx)

In [19]:
for _r in ro:
    _r.show()

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

1/2*x*diff(z(x, y), y)/(x^2*y + y^2) - w(x, y)/y + diff(w(x, y), y)

In [20]:
f1.show()
f2.show()
cf2a=reduce(f2dp, f1dp, ctx)
cf2a.show()

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

2*y*diff(z(x, y), x)/x + w(x, y)/x - 1/2*diff(z(x, y), y)/(x^2 + y) + diff(z(x, y), x, y)

In [21]:
(f2 - diff(f1, y) * y/x).expand().show()
cf=Differential_Polynomial((f2 - diff(f1, y) * y/x).expand(),ctx)
cf.show()

y*diff(w(x, y), y)/x - y*diff(w(x, y), y, y)/x + 2*y*diff(z(x, y), x)/x + diff(w(x, y), y)/x + 1/2*diff(z(x, y), y)/(x^4 + 2*x^2*y + y^2) - 1/2*diff(z(x, y), y, y)/(x^2 + y) - w(x, y)/(x*y) + 1/2*diff(z(x, y), y)/((x^2 + y)*y) + diff(z(x, y), x, y)

-(y + 1)*diff(w(x, y), y)/y - x*diff(z(x, y), x, y)/y - 1/2*(x^3 + 2*x*y)*diff(z(x, y), y)/(x^4*y^2 + 2*x^2*y^3 + y^4) + 1/2*x*diff(z(x, y), y, y)/(x^2*y + y^2) + w(x, y)/y^2 + diff(w(x, y), y, y) - 2*diff(z(x, y), x)

In [22]:
diff (f1,y).collect(diff(z, y))

-1/2*(x/((x^2 + y)*y^2) + x/((x^2 + y)^2*y))*diff(z(x, y), y) - diff(w(x, y), y)/y + 1/2*x*diff(z(x, y), y, y)/((x^2 + y)*y) + w(x, y)/y^2 + diff(w(x, y), y, y)

In [23]:
(f2 - diff(f1, y) * y/x).expand()
cf=Differential_Polynomial((f2 - diff(f1, y) * y/x).expand(),ctx)
cf.show()

-(y + 1)*diff(w(x, y), y)/y - x*diff(z(x, y), x, y)/y - 1/2*(x^3 + 2*x*y)*diff(z(x, y), y)/(x^4*y^2 + 2*x^2*y^3 + y^4) + 1/2*x*diff(z(x, y), y, y)/(x^2*y + y^2) + w(x, y)/y^2 + diff(w(x, y), y, y) - 2*diff(z(x, y), x)

In [24]:
cf2a.expression().expand().collect(diff(w,y)).collect(w(x,y)).show()

2*y*diff(z(x, y), x)/x + w(x, y)/x - 1/2*diff(z(x, y), y)/(x^2 + y) + diff(z(x, y), x, y)

In [25]:
def Autoreduce(S, context):  
    dps = list(S)
    i = 0
    while True:
        p = dps[:i+1]
        r = dps[i+1:]
        if not r:
            return dps
        newdps = []
        have_reduced = False
        for _r in r:
            rnew = reduceS(_r, p, context)
            have_reduced = have_reduced or rnew != _r
            newdps.append(rnew)
        dps = Reorder(p + [_ for _  in newdps if _ not in p], context, ascending = True)
        if not have_reduced:
            # no reduction done
            i += 1        
        else:
            # start from scratch
            i = 0            

In [26]:
def JanetBasis(S, context):
    """List of homogenous PDE's + order context"""
    if not isinstance(S, Iterable):
        # bad criterion
        S = [S]
    S = Reorder([Differential_Polynomial(s, context) for s in S], context, ascending = True)
    while 1:
        S = Autoreduce (S, context)
        #S = CompleteSystem (S, context)
        #f = FindIntegrableConditions(S, context)
        #c = ReduceIntegrabilityConditions(S, context)
        #if all ([_ == 0 for _ in c]) or not c:
        #    return S
        #S += c
        #S = Reorder (S, context)
        return S[:]

In [ ]:
checkS=JanetBasis([f3,f1,f5, f2,f4], ctx)

In [ ]:
for _ in checkS:
    _.show()

In [ ]:
checkS[0].show()

In [ ]:
checkS[1].show()

In [ ]:
checkS[2].show()

In [ ]:
checkS[3].show()

so only the last step is missing, so have to check the final critera

In [ ]:
bool(checkS[3]._p[1]._coeff == 0)

In [ ]:
g1 = Differential_Polynomial (diff(z,y,y) + diff(z,y)/(2*y), ctx)

In [ ]:
g2 = Differential_Polynomial(diff(w,x,x) + 4*y*y*diff(w,y) - 8*y*y*diff(z, x) - 8*y*w(x,y), ctx)

In [ ]:
g3 = Differential_Polynomial(diff(w,x,y) - diff(z,x,x)/2 - diff(w,x)/(2*y) - 6*y**2*diff(z,y), ctx)

In [ ]:
g4 = Differential_Polynomial(diff(w,y,y) - 2*diff(z,x,y)- diff(w,y)/(2*y) + w(x,y)/(2*y**2), ctx)

In [ ]:

res = reduceS(f4dp, [f1dp,f2dp], ctx)

In [ ]:
resdp=Differential_Polynomial (res.expression(), ctx)

In [ ]:
ex=resdp.expression().operands()

In [ ]:
ex1=ex[0].operands()

In [ ]:
ex1

In [ ]:
ex1=ex1[0]*ex1[2]

In [ ]:
ex1.expand().simplify_full()

In [ ]:
f4dp.show()

In [ ]:
f1dp.show()

In [ ]:
f2dp.show()